# Desafío Data Engineer
### Ignacio Urízar
### Agosto, 2024
- [Introducción](#intro)
  - [Supuestos](#supuestos)
- [Pregunta 1 (Q1)](#q1_pregunta)
  - [Benchmark (pandas)](#q1_benchmark)
  - [q1_memory](#q1_memory)
  - [q1_time](#q1_time)
  - [Resultados Q1](#q1_resultados)
- [Pregunta 2 (Q2)](#q2_pregunta)
  - [q2_memory](#q2_memory)
  - [q2_time](#q2_time)
  - [Resultados Q2](#q2_resultados)
- [Pregunta 3 (Q3)](#q3_pregunta)
  - [q3_memory](#q3_memory)
  - [q3_time](#q3_time)
  - [Resultados Q3](#q3_resultados)

## Introducción
<a id='intro'></a>
En este jupyter notebook se presenta el desarrollo del desafío de Data Engineer. El documento se encuentra separado en secciones según se indica en el índice más arriba.

Se presenta una sección inicial de [Supuestos](#supuestos) que tomaron en consideración para el desarrollo de las soluciones.

Cada sección incluye una descripción del código utilizado para cada solución, las diferencias entre las distintas estrategias de optimización y sus correspondientes perfiles de uso de memoria y tiempo de ejecución.

El código fuente se encuentra en el siguiente repositorio público de GitHub: [repositorio](https://github.com/urizar-ignacio/data-engineer-challenge-01).

En el README de dicho repositorio se detallan distintas alternativas de ejecución de este proyecto.

### Supuestos
<a id='supuestos'></a>

#### Ubicación del archivo con datos
Dado que el archivo con la muestra de datos que se recibió para este desafío supera el límite de tamaño aceptado por GitHub, es necesario depositarlo manualmente en la carpeta `/app/data/` para que las funciones lo puedan leer sin problemas.

#### Ambiente de ejecución
Este proyecto está desarrollado para ser ejecutado de forma local, ya sea en un computador personal o en una máquina virtual. Se recomienda usar la ejecución con Docker descrita en el README del repositorio para tener ambientes idénticos. Si se opta por una ejecución no contenerizada, se debe contar con una versión de Python compatible con las librerías listadas en el archivo `/app/requirements.txt`.

#### Estructura de carpetas
Se espera que la estructura de carpetas y distribución de archivos no sea modificada (salvo por la necesidad de incluir manualmente el archivo con la muestra de datos, como se mencionó anteriormente). Para que las importaciones funcionen correctamente se debe mantener la distribución de archivos tal como se presenta en el repositorio:

```
app/
  data/
    <archivo_con_data>.json
  src/
    challenge.ipynb
    q1_memory.py
    q1_time.py
    q2_memory.py
    q2_time.py
    q3_memory.py
    q3_time.py
  requirements.txt
```

#### Flujo de ejecución
Todas las soluciones propuestas tendrán el mismo flujo de ejecución:
- Explicación de la implementación.
- Ejecución directa, para ver el resultado generado.
- Perfilado de tiempo de ejecución.
- Perfilado de uso de memoria.

#### Perfiladores utilizados
- Para el perfilado de tiempo de ejecución se utiliza la librería `cProfile`.
- Para el perfilado de uso de memoria se utiliza la librería `memory_profiler` y el magic command `%mprun`.

#### Reinicio del kernel
Notar que entre las ejecuciones de cada solución propuesta se reinició el kernel de ejecución. Esto para asegurar que el perfilado de uso de memoria no se vea afectado a calcular memoria utilizada al momento de definición del método por librerías importadas en otras soluciones. Si bien la parte importante son los aumentos de memoria dentro de cada método, esta distinción se vuelve importante si se quiere analizar el comportamiento completo e independiente de cada solución.

#### ¿Qué es un emoji?
En la solución de la pregunta 2 (Q2) se pide un conteo de "emojis". Para esto es necesario determinar qué se considera un emoji en el alcance de este proyecto. En el contenido del texto de los tweets disponibles en la data de muestra, hay campos en formato `unicode`, algunos representan emojis otros representan caracteres de otras lenguas. Para determinar cuáles de estos caracteres especiales son emojis se utilizó el diccionario disponible en la librería de python `emoji (version 2.12.1)`. Por lo que si un emoji no está incluído en ese diccionario, no aparecerá contabilizado como tal.

#### Usernames en X (ex-Twitter)
La creación de usernames tiene algunas restricciones básicas que se deben considerar según aparece documentado en [help.x.com](https://help.x.com/en/managing-your-account/x-username-rules). Un username debe tener entre 4 y 15 caracteres, solo alfanuméricos y guión bajo. Esto se tendrá en consideración al momento de buscar menciones en la pregunta 3 (Q3), asegurando así que no se consideren para el conteo menciones como `@a`, `@123`, `@ho.la`, etc.

## Pregunta 1 (Q1)
<a id='q1_pregunta'></a>

Se requiere obtener los 10 días con más tweets. De estos 10 días se necesita identificar al usuario que realizó más posts.

### Benchmark (pandas)
<a id='q1_benchmark'></a>

Antes de comenzar con los procesos de optimización, se decidió resolver la pregunta 1 usando herramientas recurrentes en flujos de data engineering y analytics. Para este caso se utilizó `pandas`, con la idea de tener un punto de comparación base sobre el cual buscar rutas de optimización de tiempo y recursos.

Para esto se incluyó un archivo adicional: `app/src/q1_benchmark.py`.

In [1]:
from q1_benchmark import q1_benchmark
file_path = "farmers-protest-tweets-2021-2-4.json"

**Método: q1_benchmark(file_path: str)**

Toma la ruta del archivo de datos como parámetro. Abre el archivo y lo lee línea por línea y deserializa cada registro como un diccionario y los almacena en una lista. Esa lista de diccionarios es usada para crear un `DataFrame` de `pandas`. Se procesan la columna `"date"` para eliminar la parte que contiene la hora y obtiene el campo `"username"` desde el diccionario anidado `"user"`.

Con el `DataFrame` procesado, se agrupa por `"date"` para obtener el top 10 de días con más tweets y luego se agrupa por `"date"`y `"username"` para obtener los usuarios con más tweets por cada día.

Estos resultados se cruzan para obtener el usuario con más tweets por cada días del top de días con más tweets "over-all".

El `DataFrame` resultante se transforma en una lista de tuplas y es retornado como resultado.

In [2]:
# ejecución directa
q1_benchmark(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

El resultado es una lista de 10 elementos con el formato y tipo esperado `List[Tuple[datetime.date, str]]`.

In [3]:
# perfilado de tiempo de ejecución
import cProfile
cProfile.run('q1_benchmark(file_path)')

         1871333 function calls (1871106 primitive calls) in 6.685 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       78    0.000    0.000    0.000    0.000 <frozen abc>:117(__instancecheck__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.052    0.000    0.113    0.000 <frozen codecs>:319(decode)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1207(_handle_fromlist)
        1    0.066    0.066    6.685    6.685 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 __init__.py:225(compile)
        4    0.000    0.000    0.000    0.000 __init__.py:272(_compile)
   117407    0.110    0.000    4.926    0.000 __init__.py:299(loads)
       49    0.000    0.000    0.000    0.000 __init__.py:34(using_copy_on_write)
       14    0.000    0.000    0.000    0.000 __init__

La ejecución tomó un total de `6.685` segundos y se puede ver que más tiempo tomó fue la deserialización de los registros a diccionarios usando el método `loads()`, que tomó un `4.926` segundos.

In [4]:
# perfilado de uso de memoria
%load_ext memory_profiler
%mprun -f q1_benchmark q1_benchmark(file_path)

Filename: /app/src/q1_benchmark.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    402.9 MiB    402.9 MiB           1   def q1_benchmark(file_path: str) -> List[Tuple[datetime.date, str]]:
     7    402.9 MiB      0.0 MiB           1       records = []
     8   1294.7 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
     9   1294.7 MiB     28.6 MiB      117408           for line in jfile:
    10   1294.7 MiB    863.1 MiB      117407               records.append(json.loads(line))
    11   1295.8 MiB      1.1 MiB           1       df = DataFrame.from_records(records)
    12   1303.0 MiB      7.2 MiB           1       df["date"] = df["date"].str[:10]
    13   1303.0 MiB      0.0 MiB      234815       df["username"] = df["user"].apply(lambda x: x["username"])
    14                                         
    15   1303.0 MiB      0.0 MiB           1       df_by_day = df.groupby(["date"])["date"].count().nlargest(10).reset_index(name

Esta solución utilizó un total de `1303.9 MiB` de memoria. El punto de mayor incremento es la acumulación de registros en forma de diccionarios en la lista de elementos leídos desde el archivo, generando un incremento de `880.5 MiB`. También es importante notar el uso de recursos al momento de la definición del método: `402.9 MiB` utilizados solo por las dependencias del método.

### q1_memory
<a id='q1_memory'></a>

`app/src/q1_memory.py`

Este método apunta a optimizar el uso de memoria. Para esto, lo primero que se modificó respecto del **benchmark** fue el uso de `pandas`, reemplazándolo por librerías estándar de Python. Otro punto de mejora en términos de uso de memoria fue evitar el uso de diccionarios para almacenar los registros, por lo que se usó una lista de tuplas que solo almacena las partes necesarias de los datos a analizar.

**NOTA:** El kernel se reinició en este punto para limpiar cualquier `import` anterior no utilizado en este punto.

In [1]:
from q1_memory import q1_memory

file_path = "farmers-protest-tweets-2021-2-4.json"

**Método: q1_memory(file_path: str)**

Toma la ruta del archivo de datos como parámetro. Abre el archivo y lo lee línea por línea y deserializa cada registro como un diccionario y almacena solo los datos necesarios en una lista de tuplas. Se ordena la lista de tuplas por el campo de fecha y se agrupan para obtener la lista de los 10 días con más tweets.

Luego se ordena por fecha y username, y se agrupa para obtener los usuarios con mayor cantidad de tweets por día.

Finalmente se itera sobre la lista de los 10 días con más tweets y por cada uno se obtiene el usuario con más tweets correspondiente. Los resultado son almacenados en una lista de tuplas que es retornada como resultado.

Se agrega una serie de comandos para eliminar las listas generadas y asegurar la liberación de memoria, aunque más adelante, en el perfilado de uso de memoria se evidencia que no aporta sustancialmente.

In [2]:
# ejecución directa
q1_memory(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

El resultado es una lista de 10 elementos con el formato y tipo esperado `List[Tuple[datetime.date, str]]`.

La lista es igual que la generada por el método `q1_benchmark()`.

In [3]:
# perfilado de tiempo de ejecución
import cProfile
cProfile.run('q1_memory(file_path)')

         2473223 function calls in 3.665 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.042    0.000    0.098    0.000 <frozen codecs>:319(decode)
        1    0.004    0.004    3.665    3.665 <string>:1(<module>)
   117407    0.105    0.000    2.348    0.000 __init__.py:299(loads)
       10    0.000    0.000    0.000    0.000 _strptime.py:26(_getlang)
       10    0.000    0.000    0.001    0.000 _strptime.py:309(_strptime)
       10    0.000    0.000    0.001    0.000 _strptime.py:565(_strptime_datetime)
   117407    0.161    0.000    2.204    0.000 decoder.py:332(decode)
   117407    1.929    0.000    1.929    0.000 decoder.py:343(raw_decode)
       10    0.000    0.000    0.000    0.000 locale.py:396(normalize)
       10    0.000    0.000    0.000    0.00

La ejecución tomó un total de `3.665` segundos. Un `45%` más rápido que el método base `q1_benchmark`.

In [4]:
# perfilado de uso de memoria
%load_ext memory_profiler
%mprun -f q1_memory q1_memory(file_path)

Filename: /app/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7    108.5 MiB    108.5 MiB           1   def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
     8    108.5 MiB      0.0 MiB           1       records = []
     9    112.3 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
    10    112.3 MiB      0.2 MiB      117408           for line in jfile:
    11    112.3 MiB      3.6 MiB      117407               jline = loads(line)
    12    112.3 MiB      0.0 MiB      117407               records.append((jline["date"][:10], jline["user"]["username"]))
    13                                             
    14    112.5 MiB      0.1 MiB      234815       ordered_by_date_records = sorted(records, key=lambda x: x[0])
    15    112.5 MiB      0.0 MiB      234830       count_by_date = [(k, len(list(g))) for k, g in (groupby(ordered_by_date_records, lambda x: x[0]))]
    16    112.5 MiB      0.0 MiB        

Esta solución utilizó un total de `118.8 MiB` de memoria. El punto de mayor incremento es la deserialización de elementos leídos desde el archivo, generando un incremento de `3.6 MiB` y la generación de la lista de registros ordenado por fecha y username, que generó un incremento de `4.3 MiB`. También es importante notar el uso de recursos al momento de la definición del método: `108.5 MiB` utilizados solo por las dependencias del método.

El uso total de memoria fue `90,9%` mejor que el método base `q1_benchmark`.

En este perfilado se puede ver que los comandos de eliminación de variables `del` solo generan una reducción del uso de memoria respecto de una variable. Para el resto no genera diferencia.

### q1_time
<a id='q1_time'></a>

`app/src/q1_time.py`

Este método apunta a optimizar el tiempo de ejecución. Para esto, al igual que en `q1_memory` se priorizó el uso de librerías estándar de Python. Otro punto de mejora en términos de uso de tiempo de ejecución fue evitar el uso de deserialización de json, por lo que se usó expresiones regulares para obtener la data necesaria para el análisis.

**NOTA:** El kernel se reinició en este punto para limpiar cualquier `import` anterior no utilizado en este punto.

In [1]:
from q1_time import q1_time

file_path = "farmers-protest-tweets-2021-2-4.json"

**Método: q1_time(file_path: str)**

Toma la ruta del archivo de datos como parámetro. Abre el archivo, lo lee línea por línea, obtiene la información necesaria de cada registro mediante dos búsquedas con expresiones regulares y almacena solo los datos necesarios en una lista de diccionarios. Se ordena la lista de tuplas por el campo de fecha y se agrupan para obtener la lista de los 10 días con más tweets.

Luego se ordena por fecha y username, y se agrupa para obtener los usuarios con mayor cantidad de tweets por día.

Finalmente se itera sobre la lista de los 10 días con más tweets y por cada uno se obtiene el usuario con más tweets correspondiente. Los resultado son almacenados en una lista de tuplas que es retornada como resultado.

Se agrega una serie de comandos para eliminar las listas generadas y asegurar la liberación de memoria, aunque más adelante, en el perfilado de uso de memoria se evidencia que no aporta sustancialmente.

In [2]:
# ejecución directa
q1_time(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

El resultado es una lista de 10 elementos con el formato y tipo esperado `List[Tuple[datetime.date, str]]`.

La lista es igual que la generada por el método `q1_benchmark()` y el método `q1_memory()`.

In [4]:
# perfilado de tiempo de ejecución
import cProfile
cProfile.run('q1_time(file_path)')

         2473223 function calls in 2.948 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.032    0.000    0.080    0.000 <frozen codecs>:319(decode)
        1    0.016    0.016    2.948    2.948 <string>:1(<module>)
   234814    0.098    0.000    1.640    0.000 __init__.py:173(search)
   234814    0.097    0.000    0.130    0.000 __init__.py:272(_compile)
       10    0.000    0.000    0.000    0.000 _strptime.py:26(_getlang)
       10    0.000    0.000    0.001    0.000 _strptime.py:309(_strptime)
       10    0.000    0.000    0.001    0.000 _strptime.py:565(_strptime_datetime)
       10    0.000    0.000    0.000    0.000 locale.py:396(normalize)
       10    0.000    0.000    0.000    0.000 locale.py:479(_parse_localename)
       10    0.000    0.000    0.0

La ejecución tomó un total de `2.948` segundos. Un `56%` más rápido que el método base `q1_benchmark`, y `19%` más rápido que `q1_memory`.

In [5]:
# perfilado de uso de memoria
%load_ext memory_profiler
%mprun -f q1_time q1_time(file_path)

Filename: /app/src/q1_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    109.2 MiB    109.2 MiB           1   def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     7    109.2 MiB      0.0 MiB           1       records = []
     8    129.7 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
     9    129.7 MiB      3.0 MiB      117408           for line in jfile:
    10    129.7 MiB      0.0 MiB      117407               parsed_date = re.search('(?<=\"date\": \").+?(?=\")', line).group()[:10]
    11    129.7 MiB      3.0 MiB      117407               parsed_username = re.search('(?<=\"username\": \").+?(?=\")', line).group()
    12    129.7 MiB     14.5 MiB      117407               records.append({"date": parsed_date, "username": parsed_username})
    13                                             
    14    129.7 MiB      0.0 MiB      234815       ordered_by_date_records = sorted(records, key=lambda x: x["date"]

Esta solución utilizó un total de `136.8 MiB` de memoria. El punto de mayor incremento es la acumulación de elementos deserializados en la lista de diccionarios, generando un incremento de `14.5 MiB` y la generación de la lista de registros ordenado por fecha y username, que generó un incremento de `3.1 MiB`. También es importante notar el uso de recursos al momento de la definición del método: `109.2 MiB` utilizados solo por las dependencias del método.

El uso total de memoria fue `89,5%` mejor que el método base `q1_benchmark`. Pero también un un `15%` **peor** que `q1_memory`.

### Resultados Q1
<a id='q1_resultados'></a>

|Método | Tiempo de ejecución | Uso de memoria total |
|-------|---------------------|----------------------|
|q1_benchmark| 6.685 s| 1,303.9 MiB|
|q1_memory| 3.665 s| **118.8 MiB**|
|q1_time| **2.948 s**| 136.8 MiB|

## Pregunta 2 (Q2)
<a id='q2_pregunta'></a>

Se requiere obtener los 10 emojis más usados. Se considera cada aparición individual para el conteo, es decir, en un único post pueden aparecen múltiples emojis o múltiples apariciones del mismo.

**NOTA:** Para este caso no se utilizó un caso de benchmark con herramientas como `pandas` ya que en la pregunta anterior se demostró que no es ideal si se quiere optimizar tiempo de ejecución o uso de memoria.

### q2_memory
<a id='q2_memory'></a>

`app/src/q2_memory.py`

Este método apunta a optimizar el uso de memoria. Para esto se evita el uso de estructuras de datos innecesarias y también se evita almacenar data que no será usada downstream en el proceso.

**NOTA:** El kernel se reinició en este punto para limpiar cualquier `import` anterior no utilizado en este punto.

In [1]:
from q2_memory import q2_memory

file_path = "farmers-protest-tweets-2021-2-4.json"

**Método: q2_memory(file_path: str)**

Al leer el archivo se deserializa el registro pero solo se almacena la parte necesaria del dato, en este caso el contenido del campo `content`, en el que se buscará el uso de emojis. Luego se obtiene el listado de caracteres considerados "emojis" según el diccionario disponible en la librería `emoji`. Luego se recorre caracter por caracter todos los registros obtenidos y se evalúa si son emojis. En caso de ser un emoji, se agregan a un contador que se almacena en un diccionario, donde cada emoji encontrado es una llave y su valor es la cantidad de veces que se ha encontrado. El diccionario resultante con el conteo se transforma en una lista de tuplas, se ordena de mayor a menor y se retorna el top 10 como resultado.

In [2]:
# ejecución directa
q2_memory(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('🏻', 2080),
 ('❤', 1779),
 ('🤣', 1668),
 ('🏽', 1218),
 ('👇', 1108)]

El resultado es una lista de 10 elementos con el formato y tipo esperado `List[Tuple[str, int]]`.

In [4]:
# perfilado de tiempo de ejecución
import cProfile
cProfile.run('q2_memory(file_path)')

         1319903 function calls in 4.320 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.036    0.000    0.087    0.000 <frozen codecs>:319(decode)
        1    0.005    0.005    4.320    4.320 <string>:1(<module>)
   117407    0.102    0.000    2.259    0.000 __init__.py:299(loads)
   117407    0.157    0.000    2.120    0.000 decoder.py:332(decode)
   117407    1.858    0.000    1.858    0.000 decoder.py:343(raw_decode)
      641    0.000    0.000    0.000    0.000 q2_memory.py:16(<lambda>)
        1    1.309    1.309    4.315    4.315 q2_memory.py:5(q2_memory)
        1    0.638    0.638    2.983    2.983 q2_memory.py:7(<listcomp>)
    49772    0.050    0.000    0.050    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    4.320    4.

La ejecución tomó un total de `4.320` segundos.

In [5]:
# perfilado de uso de memoria
%load_ext memory_profiler
%mprun -f q2_memory q2_memory(file_path)

Filename: /app/src/q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     5    135.1 MiB    135.1 MiB           1   def q2_memory(file_path: str) -> List[Tuple[str, int]]:
     6    151.0 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
     7    151.0 MiB     15.9 MiB      117410           records = [json.loads(line)["content"] for line in jfile]
     8                                         
     9    151.0 MiB      0.0 MiB           1       emoji_list = EMOJI_DATA.keys()
    10    151.0 MiB      0.0 MiB           1       emoji_count = {}
    11    151.4 MiB      0.0 MiB      117408       for record in records:
    12    151.4 MiB      0.4 MiB    17151490           for char in record:
    13    151.4 MiB      0.0 MiB    17034083               if char in emoji_list:
    14    151.4 MiB      0.0 MiB       45636                   current_count = emoji_count.get(char, 0)
    15    151.4 MiB      0.0 MiB       45636               

Esta solución utilizó un total de `151.4 MiB` de memoria. El punto de mayor incremento es la acumulación de elementos deserializados en la lista de diccionarios, generando un incremento de `15.9 MiB`. También es importante notar el uso de recursos al momento de la definición del método: `135.1 MiB` utilizados solo por las dependencias del método.

### q2_time
<a id='q2_time'></a>

`app/src/q2_time.py`

Este método apunta a optimizar el uso de memoria. Para esto se utiliza la posibilidad de procesamiento paralelo que ofrece python a través de la librería estándar `threading`.

**NOTA:** El kernel se reinició en este punto para limpiar cualquier `import` anterior no utilizado en este punto.

In [1]:
from q2_time import q2_time

file_path = "farmers-protest-tweets-2021-2-4.json"

**Método: q2_time(file_path: str)**

Al leer el archivo se deserializa el registro pero solo se almacena la parte necesaria del dato, en este caso el contenido del campo `content`, en el que se buscará el uso de emojis. La lista de registros es dividida en grupos y cada grupo es procesado de forma independiente en un thread. Se utiliza una función auxiliar `count_emojis` que se aplica a cada grupo de registros.

En cada grupo se buscan los emojis por caracter y se almacenan en diccionarios donde se lleva el conteo. Cuando todos los threads han terminado de procesar se juntan los diccionarios de resultados en un `Counter` que sumará los resultados independientes.

Finalmente se transforma el counter en una lista de tuplas, se ordena y luego se toma el 10 y se retorna como resultado.

In [2]:
# ejecución directa
q2_time(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('🏻', 2080),
 ('❤', 1779),
 ('🤣', 1668),
 ('🏽', 1218),
 ('👇', 1108)]

El resultado es una lista de 10 elementos con el formato y tipo esperado `List[Tuple[str, int]]`. Es igual al resultado generado por la función `q2_memory`.

In [5]:
# perfilado de tiempo de ejecución
import cProfile
cProfile.run('q2_time(file_path)')

         1394992 function calls in 3.674 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       12    0.000    0.000    0.000    0.000 <frozen abc>:117(__instancecheck__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.037    0.000    0.087    0.000 <frozen codecs>:319(decode)
        1    0.005    0.005    3.674    3.674 <string>:1(<module>)
   117407    0.101    0.000    2.251    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 __init__.py:587(__init__)
       13    0.001    0.000    0.001    0.000 __init__.py:660(update)
       12    0.000    0.000    0.000    0.000 _weakrefset.py:39(_remove)
       12    0.000    0.000    0.000    0.000 _weakrefset.py:85(add)
   117407    0.153    0.000    2.114    0.000 decoder.py:332(decode)
   117407    1.855    0.000    1.855    0.000 decod

La ejecución tomó un total de `3.674` segundos. Un `15%` más rápido que `q2_memory`.

In [4]:
# perfilado de uso de memoria
%load_ext memory_profiler
%mprun -f q2_time q2_time(file_path)

Filename: /app/src/q2_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    18    128.5 MiB    128.5 MiB           1   def q2_time(file_path: str) -> List[Tuple[str, int]]:
    19    128.5 MiB      0.0 MiB           1       records = []
    20    139.2 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
    21    139.2 MiB      0.0 MiB      117408           for line in jfile:
    22    139.2 MiB     10.6 MiB      117407               jline = json.loads(line)
    23    139.2 MiB      0.0 MiB      117407               records.append(jline["content"])
    24                                         
    25    139.7 MiB      0.5 MiB           1       emoji_list = set(EMOJI_DATA.keys())
    26    139.7 MiB      0.0 MiB           1       threads = []
    27    139.7 MiB      0.0 MiB           1       results = []
    28                                         
    29    139.7 MiB      0.0 MiB           1       chunk_size = 10000
    30    139.

Esta solución utilizó un total de `139.8 MiB` de memoria. Ocupa un `7%` menos de memoria que el caso anterior. A pesar de que la lectura del archivo es la misma y rescata la misma cantidad de data que `q2_memory`, parece tener un incremento menor de consumo.

Curiosamente, a pesar de enfocar el esfuerzo de esta solución a la velocidad de ejecución mediante paralelización, esto parece tener también un beneficio en la liberación de recursos del proceso.

También es importante notar el uso de recursos al momento de la definición del método: `128.5 MiB` utilizados solo por las dependencias del método.

### Resultados Q2
<a id='q2_resultados'></a>

|Método | Tiempo de ejecución | Uso de memoria total |
|-------|---------------------|----------------------|
|q2_memory| 4.320 s| 151.4 MiB|
|q2_time| **3.674 s**| **139.8 MiB**|

`q2_time` resultó mejor en ambos aspectos.

## Pregunta 3 (Q2)
<a id='q3_pregunta'></a>

Se requiere obtener la lista de los 10 usernames más mencionados. Se considera cada aparición individual para el conteo, es decir, en un único post pueden aparecen múltiples menciones de distintos o el mismo username.

**NOTA:** Para este caso tampoco se utilizó un caso de benchmark con herramientas como `pandas` ya que en la pregunta anterior se demostró que no es ideal si se quiere optimizar tiempo de ejecución o uso de memoria.

### q3_memory
<a id='q3_memory'></a>

`app/src/q3_memory.py`

Este método apunta a optimizar el uso de memoria. Para esto se evita el uso de estructuras de datos innecesarias y también se evita almacenar data que no será usada downstream en el proceso usando un filtro adicional.

**NOTA:** El kernel se reinició en este punto para limpiar cualquier `import` anterior no utilizado en este punto.

In [1]:
from q3_memory import q3_memory

file_path = "farmers-protest-tweets-2021-2-4.json"

**Método: q3_memory(file_path: str)**

Al leer el archivo se deserializa el registro pero solo se almacena la parte necesaria del dato, en este caso el contenido del campo `content`, en el que se buscarán las menciones de usernames. Además, al rescatar los datos se almacenan solo aquellos registros que tengan al menos un `@` en su contenido, esto para evitar almacenar aquellos registros que se puede asegurar que no tienen ninguna mención.

Sobre la lista de registros que contienen al menos un `@` se aplica una expresión regular que lista todas las menciones de usernames válidos. Se itera sobre la lista de usernames válidos y se lleva el conteo de estos en un diccionario.

Cuando se termina de iterar sobre todos los registros y sobre todas las menciones, el diccionario de conteo se transforma en una lista de tuplas, se ordena y se retorna el top 10 como resultado.


In [2]:
# ejecución directa
q3_memory(file_path)

[('@narendramodi', 2261),
 ('@Kisanektamorcha', 1836),
 ('@RakeshTikaitBKU', 1639),
 ('@PMOIndia', 1422),
 ('@RahulGandhi', 1125),
 ('@GretaThunberg', 1046),
 ('@RaviSinghKA', 1015),
 ('@rihanna', 972),
 ('@UNHumanRights', 962),
 ('@meenaharris', 925)]

El resultado es una lista de 10 elementos con el formato y tipo esperado `List[Tuple[str, int]]`.

In [3]:
# perfilado de tiempo de ejecución
import cProfile
cProfile.run('q3_memory(file_path)')

         1144593 function calls in 2.262 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.043    0.000    0.097    0.000 <frozen codecs>:319(decode)
        1    0.002    0.002    2.262    2.262 <string>:1(<module>)
    55172    0.024    0.000    0.068    0.000 __init__.py:218(finditer)
    55172    0.022    0.000    0.030    0.000 __init__.py:272(_compile)
    55172    0.056    0.000    1.221    0.000 __init__.py:299(loads)
    55172    0.077    0.000    1.146    0.000 decoder.py:332(decode)
    55172    1.014    0.000    1.014    0.000 decoder.py:343(raw_decode)
    55172    0.084    0.000    0.097    0.000 q3_memory.py:11(<listcomp>)
    15514    0.002    0.000    0.002    0.000 q3_memory.py:16(<lambda>)
        1    0.064    0.064    2.260    2.260 q3_memor

La ejecución tomó un total de `2.262` segundos.

In [4]:
# perfilado de uso de memoria
%load_ext memory_profiler
%mprun -f q3_memory q3_memory(file_path)

Filename: /app/src/q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     5    104.1 MiB    104.1 MiB           1   def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     6    109.2 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
     7    109.2 MiB      5.1 MiB      117410           records = [json.loads(line)["content"] for line in jfile if '@' in line]
     8                                         
     9    109.2 MiB      0.0 MiB           1       mention_counter = {}
    10    113.3 MiB      0.0 MiB       55173       for record in records:
    11    113.3 MiB      2.5 MiB      266488           mentions = [mention.group() for mention in re.finditer(r"@([a-zA-Z0-9_]){4,15}\b", record)]
    12    113.3 MiB      0.0 MiB      156144           for mention in mentions:
    13    113.3 MiB      0.0 MiB      100972               current_count = mention_counter.get(mention, 0)
    14    113.3 MiB      1.6 MiB      100972     

Esta solución utilizó un total de `114.2 MiB` de memoria. El mayor incremento se debe al almacenamiento de registros que contienen un `@`, lo que genera un incremento de uso de memoria de `5.1 MiB`. También es importante notar el uso de recursos al momento de la definición del método: `104.1 MiB` utilizados solo por las dependencias del método.

### q3_time
<a id='q3_time'></a>

`app/src/q3_time.py`

Este método apunta a optimizar el uso de memoria. Para esto se utiliza la posibilidad de procesamiento paralelo que ofrece python a través de la librería estándar `threading`.

**NOTA:** El kernel se reinició en este punto para limpiar cualquier `import` anterior no utilizado en este punto.

In [1]:
from q3_time import q3_time

file_path = "farmers-protest-tweets-2021-2-4.json"

**Método: q3_time(file_path: str)**

Este método sigue la misma estructura que en el caso `q2_time`, salvo por el uso de un filtro al momento de leer los registros del archivo, asegurando que tengan al menos una vez el caracter `@`. Al leer el archivo se deserializa el registro pero solo se almacena la parte necesaria del dato, en este caso el contenido del campo `content`, en el que se buscarán las menciones de usernames. La lista de registros es dividida en grupos y cada grupo es procesado de forma independiente en un thread. Se utiliza una función auxiliar `count_mentions` que se aplica a cada grupo de registros.

En cada grupo se buscan las menciones mediante expresiones regulares y se almacenan en diccionarios donde se lleva el conteo. Cuando todos los threads han terminado de procesar se juntan los diccionarios de resultados en un `Counter` que sumará los resultados independientes.

Finalmente se transforma el counter en una lista de tuplas, se ordena y luego se toma el 10 y se retorna como resultado.

In [2]:
# ejecución directa
q3_time(file_path)

[('@narendramodi', 2261),
 ('@Kisanektamorcha', 1836),
 ('@RakeshTikaitBKU', 1639),
 ('@PMOIndia', 1422),
 ('@RahulGandhi', 1125),
 ('@GretaThunberg', 1046),
 ('@RaviSinghKA', 1015),
 ('@rihanna', 972),
 ('@UNHumanRights', 962),
 ('@meenaharris', 925)]

El resultado es una lista de 10 elementos con el formato y tipo esperado `List[Tuple[str, int]]`. Es igual al resultado generado por la función `q3_memory`.

In [3]:
# perfilado de tiempo de ejecución
import cProfile
cProfile.run('q3_time(file_path)')

         687777 function calls in 2.118 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        6    0.000    0.000    0.000    0.000 <frozen abc>:117(__instancecheck__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.034    0.000    0.084    0.000 <frozen codecs>:319(decode)
        1    0.002    0.002    2.118    2.118 <string>:1(<module>)
    55172    0.050    0.000    1.214    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 __init__.py:587(__init__)
        7    0.006    0.001    0.008    0.001 __init__.py:660(update)
        6    0.000    0.000    0.000    0.000 _weakrefset.py:39(_remove)
        6    0.000    0.000    0.000    0.000 _weakrefset.py:85(add)
    55172    0.073    0.000    1.146    0.000 decoder.py:332(decode)
    55172    1.021    0.000    1.021    0.000 decode

La ejecución tomó un total de `2.118` segundos. Un `6%` más rápido que `q3_memory`.

In [4]:
# perfilado de uso de memoria
%load_ext memory_profiler
%mprun -f q3_time q3_time(file_path)

Filename: /app/src/q3_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    18    107.4 MiB    107.4 MiB           1   def q3_time(file_path: str) -> List[Tuple[str, int]]:
    19    107.4 MiB      0.0 MiB           1       records = []
    20    111.4 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
    21    111.4 MiB      4.0 MiB      117410           records = [json.loads(line)["content"] for line in jfile if '@' in line]
    22                                         
    23    111.4 MiB      0.0 MiB           1       threads = []
    24    111.4 MiB      0.0 MiB           1       results = []
    25                                         
    26    111.4 MiB      0.0 MiB           1       chunk_size = 10000
    27    112.7 MiB      0.0 MiB           7       for i in range((len(records)//chunk_size)+1):
    28    112.3 MiB      0.0 MiB           6           task = Thread(target=count_mentions, kwargs={"record_list":records[i*ch

Esta solución utilizó un total de `114.5 MiB` de memoria. Aproximadamente `0.2%` más que `q3_memory`. También es importante notar el uso de recursos al momento de la definición del método: `107.4 MiB` utilizados solo por las dependencias del método.

### Resultados Q3
<a id='q3_resultados'></a>

|Método | Tiempo de ejecución | Uso de memoria total |
|-------|---------------------|----------------------|
|q3_memory| 2.262 s| **114.2 MiB**|
|q3_time| **2.118 s**| 114.5 MiB|

## Mejoras posibles
<a id='mejoras_posibles'></a>
- **Manejo de errores:** Se puede agregar control de errores con `try - catch` para asegurar que comportamientos no deseados rompan el flujo de estos métodos. El punto más evidente donde se puede implementar esto es en la deserialización de los registros desde el archivo. Para este ejemplo los datos estaban limpios y con buen formato, pero en un caso real es probable recibir registros malformados o incompletos. Aquellos que levanten una `exception` al momento de aplicar `json.loads` se podrían declarar en un log y saltarlos.
- **Comentarios:** Dado que se prefirió usar librerías estándar para procesar los datos, además de generadores y funciones `lambda` el código se vuelve más compacto y eficiente, pero también puede ser más difícil de leer, sobretodo para alguien con poca experiencia. Se podrían agregar documentación a través de `docstrings`.
- **Modularidad y Clean Architecture:** Dado que el código para cada caso resultó muy compacto, se decidió no utilizar módulos ni un método de desarrollo que separe la lógica en más archivos. Se prefirió reducir al mínimo la abstracción, pero en un caso real, usar métodos de desarrollo modular puede ayudar con la escalabilidad del proyecto, ya sea usando TDD, DDD o Clean Architecture.

## ¿Ejecución en la nube?
<a id='cloud'></a>
Ya que todos los métodos propuestos usan muy poca memoria (`<200 MiB`), podrían ser implementados sin problemas en soluciones serverless como `Cloud Function` de GCP o `Lambda` de AWS. Siempre y cuando se modifique la forma de leer el archivo para que lo saque de algún espacio de almacenamiento en la nube, como `GCS` o `S3`.

También se podrían usar otras herramientas para procesar los datos, como `Apache Spark` en `Dataproc` para realizar el procesamiento de forma distribuída, sin embargo, para volúmenes tan pequeños como el caso de ejemplo, sería una forma de `overkill` y probablemente termine demorando más y costando más dinero.